In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

['giba-r-data-table-simple-features-1-17-lb_train.pkl',
 'eem_train.pkl',
 'coulomb-interaction-speed-up_train.pkl',
 'dataset-with-number-of-bonds-between-atoms_test.pkl',
 'keras-neural-net-for-champs_train.pkl',
 'brute-force-feature-engineering_test.pkl',
 'angles-and-distances_test.pkl',
 'giba-r-data-table-simple-features-0-991-lb_test.pkl',
 'keras-neural-net-for-champs_test.pkl',
 'brute-force-feature-engineering-mini_train.pkl',
 'laplaction_matrix_test.pkl',
 'dataset-with-number-of-bonds-between-atoms_train.pkl',
 'coulomb-interaction-speed-up_test.pkl',
 'bonds-from-structure-data_train.pkl',
 'laplaction_matrix_train.pkl',
 'giba-r-data-table-simple-features-0-991-lb_train.pkl',
 'eem_test.pkl',
 'brute-force-feature-engineering_train.pkl',
 'molecule-with-openbabel_train.pkl',
 'molecular-properties-eda-and-models_train.pkl',
 'brute-force-feature-engineering-mini_test.pkl',
 'eachtype_train.pkl',
 'molecular-properties-eda-and-models_test.pkl',
 'bonds-from-structure-dat

In [5]:
mytrial = []

In [ ]:
"""
y=scalar_coupling_constant, type=0
257
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=1
454
{'columns': ['min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=2
681
{'columns': ['tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=3
139
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=4
219
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=5
299
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=6
379
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=scalar_coupling_constant, type=7
449
{'columns': ['max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
"""

y=scalar_coupling_constant, type=0
257
{'columns': ['cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dis

In [6]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant']:
    for t in np.arange(3,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=scalar_coupling_constant, type=3
139
{'columns': ['tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_dis

In [6]:
def objective(trial):
        
    learning_rate = trial.suggest_uniform('learning_rate', .01, .5)
    feature_fraction = trial.suggest_uniform('feature_fraction', .6, 1)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.6, 1)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 800)
    lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-6, 1e2)
    lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-6, 1e2)
    max_bin = trial.suggest_int('max_bin', 10, 100)
    num_leaves = trial.suggest_int('num_leaves', 4, 512)
        
    args = columns_list[t].copy()
    args['model']['init']={
                'learning_rate':learning_rate,
                'feature_fraction':feature_fraction,
                'bagging_fraction':bagging_fraction,
                'min_data_in_leaf':min_data_in_leaf,
                'lambda_l1':lambda_l1,
                'lambda_l2':lambda_l2,
                'max_bin':max_bin,
                'num_leaves':num_leaves,
                'n_jobs':16
    }
    
    df_his, df_feature_importances, df_valid_pred, df_test_pred =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), args, f'modeling for {t}', df_test=None, trial=mytrial, is_output_feature_importance=False, trial_level=1)
    val_metric_mean = np.mean(df_his.valid)
    return val_metric_mean



In [8]:
# df_trial = pd.DataFrame(mytrial)
# df_trial[df_trial['message']=='tune hyperparam'][['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff']].sort_values(by=['val_metric_mean']).head()

In [6]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [7]:
oof_file_folder =  '../../data/oof'
[f for f in os.listdir(oof_file_folder) if f.startswith('lgbm')]

['lgbm_dso_train.pkl',
 'lgbm_sd_test.pkl',
 'lgbm_fc_train.pkl',
 'lgbm_pso_train.pkl',
 'lgbm_pso_test.pkl',
 'lgbm_fc_test.pkl',
 'lgbm_dso_test.pkl',
 'lgbm_sd_train.pkl']

In [8]:
df_oof_train = pd.DataFrame()
df_oof_test = pd.DataFrame()
for f in [f for f in os.listdir(oof_file_folder) if f.startswith('lgbm')]:
    feat_name = f.split('_')[1]
    if 'train' in f:
        df_oof_i = pd.read_pickle(f'{oof_file_folder}/{f}')[['id', feat_name]].rename(columns={'id':'index', feat_name:f'oof_{feat_name}'})
        df_oof_train = pd.concat([df_oof_train, df_oof_i], axis=1)
    if 'test' in f:
        df_oof_i = pd.read_pickle(f'{oof_file_folder}/{f}')[['id', feat_name]].rename(columns={'id':'index', feat_name:f'oof_{feat_name}'})
        df_oof_test = pd.concat([df_oof_test, df_oof_i], axis=1)

In [9]:
df_train = pd.concat([df_train, df_oof_train[['oof_sd','oof_fc','oof_pso','oof_dso']]], axis=1)
df_test = pd.concat([df_test, df_oof_test[['oof_sd','oof_fc','oof_pso','oof_dso']]], axis=1)

In [10]:
columns_list = [{'columns': ['oof_fc','cos_center1', 'adC3', 'dist_H_4_y', 'inv_dist1R', 'eem_1', 'dist_C_0_y', 'dist_to_type_std', 'cos_f0', 'molecule_dist_min', 'inv_dist1', 'eem_0', 'molecule_atom_index_0_dist_max_div', 'molecule_type_dist_std_diff', 'tertiary_angle_0', 'tertiary_distance_5', 'dist_O_1_x', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_4_y', 'dist_to_type_mean', 'inv_distPR', 'inv_dist1E', 'adC1', 'dist_C_0_x', 'dist_H_1_x', 'dist_O_0_x', 'adN1', 'dist_to_type_1_mean', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'dist_H_3_x', 'tertiary_angle_1', 'linkM0', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'molecule_type_dist_mean_diff', 'dist_to_type_0_mean', 'dist_C_3_x', 'tertiary_distance_4', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'dist_H_0_y', 'tertiary_distance_0', 'dist_C_2_x', 'tertiary_angle_5', 'dist_H_0_x', 'coulomb_H.y', 'tertiary_distance_1', 'dist_H_2_y', 'dist_O_1_y', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'distC1', 'dist_xyz', 'dist_C_2_y', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'distC0', 'vander_H.x', 'dist_H_2_x', 'adC2', 'yukawa_H.y', 'molecule_atom_index_1_dist_min_diff', 'cos_center0', 'dist_C_4_x', 'dist_N_0_x', 'atom_index_farthest_0', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','min_molecule_atom_1_dist_xyz', 'tertiary_distance_1', 'adC3', 'dist_H_2_y', 'inv_dist1R', 'dist_C_0_y', 'cos_f0', 'atom_1_bond_lengths_std', 'inv_dist1', 'eem_0', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'cos_center0_center1', 'tertiary_angle_0', 'tertiary_distance_5', 'tertiary_distance_3', 'tertiary_distance_2', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'distance_center0', 'distN0', 'vander_N.x', 'inv_distPR', 'tertiary_angle_2', 'tertiary_atom_0', 'dist_H_3_y', 'adC4', 'dist_C_0_x', 'dist_H_1_x', 'tertiary_angle_3', 'dist_O_0_x', 'dist_C_1_y', 'atom_1_bond_lengths_max', 'molecule_atom_index_1_dist_max_div', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'dist_C_1_x', 'vander_H.x', 'dist_to_type_0_mean', 'inv_distP', 'dist_C_3_x', 'tertiary_distance_4', 'bond_atom', 'max_molecule_atom_0_dist_xyz', 'inv_dist0R', 'vander_H.y', 'dist_N_1_x', 'dist_H_0_y', 'yukawa_H.y', 'tertiary_distance_0', 'yukawa_H.x', 'dist_C_2_x', 'linkN', 'dist_N_0_x', 'dist_H_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','tertiary_distance_6', 'adC3', 'inv_dist1R', 'eem_1', 'cos_f0', 'dist_C_0_y', 'cos_c1', 'yukawa_O.x', 'inv_dist1', 'tertiary_atom_2', 'molecule_atom_index_0_dist_max_div', 'tertiary_distance_2', 'tertiary_angle_0', 'dist_O_1_x', 'tertiary_distance_3', 'mean_molecule_atom_0_dist_xyz', 'inv_distPR', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'dist_C_0_x', 'dist_H_1_x', 'adC1', 'dist_O_0_x', 'molecule_atom_index_0_dist_max_diff', 'adN1', 'atom_index_1_cycle_size_mean', 'dist_N_0_y', 'tertiary_angle_1', 'tertiary_angle_4', 'dist_C_1_x', 'distance_farthest_0', 'inv_dist0', 'dist_C_3_x', 'atom_0_bond_lengths_max', 'atom_1_bond_lengths_min', 'bond_atom', 'inv_dist0R', 'dist_H_0_y', 'molecule_atom_index_1_dist_min_div', 'dist_C_2_x', 'cos_c0', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_2_y', 'tertiary_atom_3', 'atom_1_n_bonds', 'atom_1_bond_lengths_std', 'dist_O_0_y', 'vander_O.y', 'dist_C_3_y', 'atom_1_bond_lengths_mean', 'dist_C_2_y', 'yukawa_O.y', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_2', 'tertiary_atom_0', 'tertiary_atom_1', 'tertiary_angle_3', 'max_molecule_atom_1_dist_xyz', 'atom_1_bond_lengths_max', 'dist_C_1_y', 'distC0', 'adC2', 'tertiary_atom_4', 'cos_c0_c1', 'atom_index_1_n_cycle', 'dist_N_0_x', 'dist_H_1_y'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_0_dist_max_div', 'adC3', 'dist_C_3_y', 'tertiary_angle_1', 'yukawa_H.y', 'cos_f0_f1', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'dist_C_0_x', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_std', 'adC1', 'tertiary_distance_1', 'dist_H_0_y', 'molecule_dist_min', 'max_distance_y', 'inv_distPE', 'dist_xyz', 'eem_0', 'dist_O_0_x', 'dist_to_type_mean', 'cos_c0_c1', 'cos_c0', 'adN1', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'inv_distP', 'molecule_atom_index_0_dist_mean_diff', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_H_1_y', 'dist_C_3_x', 'dist_H_2_y', 'dist_H_3_y', 'link0', 'yukawa_H.x', 'dist_C_2_x', 'dist_N_0_y', 'dist_to_type_0_mean', 'dist_N_0_x', 'eem_1', 'tertiary_angle_3', 'distance_c1', 'dist_H_3_x', 'tertiary_distance_3', 'cos_f0', 'cos_f1', 'tertiary_atom_2'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['oof_fc','max_molecule_atom_0_dist_xyz', 'tertiary_distance_2', 'dist_C_0_y', 'dist_H_1_x', 'molecule_atom_index_0_dist_min_diff', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_x', 'adC3', 'distN0', 'atom_0_bond_lengths_max', 'dist_C_3_y', 'tertiary_angle_1', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'cos_c1', 'adC2', 'molecule_atom_index_1_dist_max_div', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_x', 'adC1', 'tertiary_distance_1', 'atom_1_bond_lengths_std', 'dist_H_0_y', 'dist_xyz', 'dist_O_0_x', 'inv_distPR', 'cos_c0_c1', 'cos_c0', 'tertiary_angle_0', 'tertiary_distance_4', 'dist_H_0_x', 'dist_C_1_x', 'tertiary_atom_1', 'tertiary_angle_2', 'mean_molecule_atom_0_dist_xyz', 'dist_C_2_y', 'dist_C_3_x', 'atom_1_n_bonds', 'bond_atom', 'dist_C_2_x', 'dist_N_0_y', 'dist_N_0_x', 'atom_index_1_cycle_size_mean', 'min_molecule_atom_0_dist_xyz', 'tertiary_angle_3', 'tertiary_distance_3', 'cos_f0'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}]

In [13]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist()[-2:]:
    study = optuna.create_study()
    study.optimize(objective, n_trials=200)
    print(study.best_params)

[I 2019-07-23 22:02:21,710] Finished trial#0 resulted in value: 0.45763091003604384. Current best value is 0.45763091003604384 with parameters: {'learning_rate': 0.04288348363549207, 'feature_fraction': 0.6680365117708598, 'bagging_fraction': 0.9823890143227101, 'min_data_in_leaf': 318, 'lambda_l1': 7.981108956877393, 'lambda_l2': 2.9024199222231738e-05, 'max_bin': 80, 'num_leaves': 104}.
[I 2019-07-23 22:03:27,086] Finished trial#1 resulted in value: 0.443645838695506. Current best value is 0.443645838695506 with parameters: {'learning_rate': 0.04188563553270952, 'feature_fraction': 0.7582337302232606, 'bagging_fraction': 0.8041247466402766, 'min_data_in_leaf': 239, 'lambda_l1': 74.66621325054395, 'lambda_l2': 2.583841052997335e-05, 'max_bin': 70, 'num_leaves': 186}.
[I 2019-07-23 22:04:13,745] Finished trial#2 resulted in value: 0.43019612254387873. Current best value is 0.43019612254387873 with parameters: {'learning_rate': 0.2184100069423644, 'feature_fraction': 0.7261584613727758,

[I 2019-07-23 22:56:19,480] Finished trial#40 resulted in value: 0.4174382007423598. Current best value is 0.35374619816415975 with parameters: {'learning_rate': 0.49343735061513005, 'feature_fraction': 0.9984124416758248, 'bagging_fraction': 0.8766954088820053, 'min_data_in_leaf': 442, 'lambda_l1': 0.0010462015118279488, 'lambda_l2': 15.947298598549185, 'max_bin': 33, 'num_leaves': 488}.
[I 2019-07-23 22:57:54,286] Finished trial#41 resulted in value: 0.35632795768495434. Current best value is 0.35374619816415975 with parameters: {'learning_rate': 0.49343735061513005, 'feature_fraction': 0.9984124416758248, 'bagging_fraction': 0.8766954088820053, 'min_data_in_leaf': 442, 'lambda_l1': 0.0010462015118279488, 'lambda_l2': 15.947298598549185, 'max_bin': 33, 'num_leaves': 488}.
[I 2019-07-23 22:59:27,903] Finished trial#42 resulted in value: 0.37606572143563965. Current best value is 0.35374619816415975 with parameters: {'learning_rate': 0.49343735061513005, 'feature_fraction': 0.998412441

[I 2019-07-23 23:55:12,609] Finished trial#80 resulted in value: 0.39102057653963357. Current best value is 0.34583010689088345 with parameters: {'learning_rate': 0.4452294569445435, 'feature_fraction': 0.9400434899485425, 'bagging_fraction': 0.8731336075370013, 'min_data_in_leaf': 284, 'lambda_l1': 1.339838226173618e-05, 'lambda_l2': 0.05455812036713921, 'max_bin': 89, 'num_leaves': 501}.
[I 2019-07-23 23:56:24,229] Finished trial#81 resulted in value: 0.4161882307921636. Current best value is 0.34583010689088345 with parameters: {'learning_rate': 0.4452294569445435, 'feature_fraction': 0.9400434899485425, 'bagging_fraction': 0.8731336075370013, 'min_data_in_leaf': 284, 'lambda_l1': 1.339838226173618e-05, 'lambda_l2': 0.05455812036713921, 'max_bin': 89, 'num_leaves': 501}.
[I 2019-07-23 23:58:09,240] Finished trial#82 resulted in value: 0.3482449707550205. Current best value is 0.34583010689088345 with parameters: {'learning_rate': 0.4452294569445435, 'feature_fraction': 0.94004348994

[I 2019-07-24 00:52:12,496] Finished trial#120 resulted in value: 0.3589275205823105. Current best value is 0.34583010689088345 with parameters: {'learning_rate': 0.4452294569445435, 'feature_fraction': 0.9400434899485425, 'bagging_fraction': 0.8731336075370013, 'min_data_in_leaf': 284, 'lambda_l1': 1.339838226173618e-05, 'lambda_l2': 0.05455812036713921, 'max_bin': 89, 'num_leaves': 501}.
[I 2019-07-24 00:53:40,053] Finished trial#121 resulted in value: 0.3620366269170163. Current best value is 0.34583010689088345 with parameters: {'learning_rate': 0.4452294569445435, 'feature_fraction': 0.9400434899485425, 'bagging_fraction': 0.8731336075370013, 'min_data_in_leaf': 284, 'lambda_l1': 1.339838226173618e-05, 'lambda_l2': 0.05455812036713921, 'max_bin': 89, 'num_leaves': 501}.
[I 2019-07-24 00:55:10,651] Finished trial#122 resulted in value: 0.3855177087014403. Current best value is 0.34583010689088345 with parameters: {'learning_rate': 0.4452294569445435, 'feature_fraction': 0.940043489

[I 2019-07-24 01:57:23,608] Finished trial#160 resulted in value: 0.34621511601500543. Current best value is 0.34532627890588197 with parameters: {'learning_rate': 0.4540923930379812, 'feature_fraction': 0.971839776738276, 'bagging_fraction': 0.6672418499150679, 'min_data_in_leaf': 534, 'lambda_l1': 0.08926785662993982, 'lambda_l2': 2.835575484786637e-05, 'max_bin': 92, 'num_leaves': 509}.
[I 2019-07-24 01:59:26,187] Finished trial#161 resulted in value: 0.3482146744100361. Current best value is 0.34532627890588197 with parameters: {'learning_rate': 0.4540923930379812, 'feature_fraction': 0.971839776738276, 'bagging_fraction': 0.6672418499150679, 'min_data_in_leaf': 534, 'lambda_l1': 0.08926785662993982, 'lambda_l2': 2.835575484786637e-05, 'max_bin': 92, 'num_leaves': 509}.
[I 2019-07-24 02:01:30,778] Finished trial#162 resulted in value: 0.34637864499508597. Current best value is 0.34532627890588197 with parameters: {'learning_rate': 0.4540923930379812, 'feature_fraction': 0.971839776

...

[I 2019-07-24 03:06:21,673] Finished trial#40 resulted in value: 0.13350644721061916. Current best value is 0.12264346439596385 with parameters: {'learning_rate': 0.3332458718876652, 'feature_fraction': 0.9608332101950607, 'bagging_fraction': 0.6867244878863739, 'min_data_in_leaf': 200, 'lambda_l1': 0.6154920556313904, 'lambda_l2': 0.15406648043476537, 'max_bin': 45, 'num_leaves': 268}.
[I 2019-07-24 03:06:29,496] Finished trial#41 resulted in value: 0.1220179946252536. Current best value is 0.1220179946252536 with parameters: {'learning_rate': 0.3895202609429864, 'feature_fraction': 0.9055797727597551, 'bagging_fraction': 0.7014300922015038, 'min_data_in_leaf': 246, 'lambda_l1': 0.0012258663759606144, 'lambda_l2': 0.005035950486205536, 'max_bin': 41, 'num_leaves': 317}.
[I 2019-07-24 03:06:38,171] Finished trial#42 resulted in value: 0.12076795893637855. Current best value is 0.12076795893637855 with parameters: {'learning_rate': 0.3700089516134344, 'feature_fraction': 0.9070764829408

[I 2019-07-24 03:10:58,433] Finished trial#80 resulted in value: 0.13688708933457028. Current best value is 0.12076795893637855 with parameters: {'learning_rate': 0.3700089516134344, 'feature_fraction': 0.9070764829408015, 'bagging_fraction': 0.7108700412312582, 'min_data_in_leaf': 255, 'lambda_l1': 0.0011187016971889543, 'lambda_l2': 0.004979923811595772, 'max_bin': 39, 'num_leaves': 451}.
[I 2019-07-24 03:11:06,677] Finished trial#81 resulted in value: 0.1231873241883202. Current best value is 0.12076795893637855 with parameters: {'learning_rate': 0.3700089516134344, 'feature_fraction': 0.9070764829408015, 'bagging_fraction': 0.7108700412312582, 'min_data_in_leaf': 255, 'lambda_l1': 0.0011187016971889543, 'lambda_l2': 0.004979923811595772, 'max_bin': 39, 'num_leaves': 451}.
[I 2019-07-24 03:11:12,883] Finished trial#82 resulted in value: 0.15507675719243716. Current best value is 0.12076795893637855 with parameters: {'learning_rate': 0.3700089516134344, 'feature_fraction': 0.90707648

[I 2019-07-24 03:16:01,829] Finished trial#120 resulted in value: 0.25362318637010006. Current best value is 0.12061738882984822 with parameters: {'learning_rate': 0.20542946038564086, 'feature_fraction': 0.9182870727324925, 'bagging_fraction': 0.9687362590794517, 'min_data_in_leaf': 208, 'lambda_l1': 8.03803683915087e-06, 'lambda_l2': 0.023141014344723734, 'max_bin': 80, 'num_leaves': 399}.
[I 2019-07-24 03:16:08,548] Finished trial#121 resulted in value: 0.13675682153700003. Current best value is 0.12061738882984822 with parameters: {'learning_rate': 0.20542946038564086, 'feature_fraction': 0.9182870727324925, 'bagging_fraction': 0.9687362590794517, 'min_data_in_leaf': 208, 'lambda_l1': 8.03803683915087e-06, 'lambda_l2': 0.023141014344723734, 'max_bin': 80, 'num_leaves': 399}.
[I 2019-07-24 03:16:15,991] Finished trial#122 resulted in value: 0.13431504001386477. Current best value is 0.12061738882984822 with parameters: {'learning_rate': 0.20542946038564086, 'feature_fraction': 0.918

[I 2019-07-24 03:21:10,274] Finished trial#160 resulted in value: 0.12598929824484742. Current best value is 0.11802456126506859 with parameters: {'learning_rate': 0.3069006560814881, 'feature_fraction': 0.9963388008472429, 'bagging_fraction': 0.8793632084474166, 'min_data_in_leaf': 217, 'lambda_l1': 1.1982777608084696, 'lambda_l2': 26.943766780064788, 'max_bin': 80, 'num_leaves': 461}.
[I 2019-07-24 03:21:15,811] Finished trial#161 resulted in value: 0.1364250229860708. Current best value is 0.11802456126506859 with parameters: {'learning_rate': 0.3069006560814881, 'feature_fraction': 0.9963388008472429, 'bagging_fraction': 0.8793632084474166, 'min_data_in_leaf': 217, 'lambda_l1': 1.1982777608084696, 'lambda_l2': 26.943766780064788, 'max_bin': 80, 'num_leaves': 461}.
[I 2019-07-24 03:21:23,598] Finished trial#162 resulted in value: 0.13476714912442872. Current best value is 0.11802456126506859 with parameters: {'learning_rate': 0.3069006560814881, 'feature_fraction': 0.996338800847242

{'learning_rate': 0.3069006560814881, 'feature_fraction': 0.9963388008472429, 'bagging_fraction': 0.8793632084474166, 'min_data_in_leaf': 217, 'lambda_l1': 1.1982777608084696, 'lambda_l2': 26.943766780064788, 'max_bin': 80, 'num_leaves': 461}


In [11]:
param_model_init_list = [
{'learning_rate': 0.4847973878400747, 'feature_fraction': 0.951715013033655, 'bagging_fraction': 0.6640125849570521, 'min_data_in_leaf': 437, 'lambda_l1': 0.0001680598586558378, 'lambda_l2': 5.156758691503555e-06, 'max_bin': 56, 'num_leaves': 490},
{'learning_rate': 0.4041014002815698, 'feature_fraction': 0.9333037289627406, 'bagging_fraction': 0.7450389220980993, 'min_data_in_leaf': 243, 'lambda_l1': 3.7722915702800837e-06, 'lambda_l2': 4.243139074965437e-05, 'max_bin': 100, 'num_leaves': 506},
{'learning_rate': 0.3864250628338235, 'feature_fraction': 0.934428347076253, 'bagging_fraction': 0.8030642265243596, 'min_data_in_leaf': 203, 'lambda_l1': 0.022599448912350148, 'lambda_l2': 4.0097557007881285e-05, 'max_bin': 63, 'num_leaves': 301},
{'learning_rate': 0.3310621175605656, 'feature_fraction': 0.9096874372780241, 'bagging_fraction': 0.8461946729271688, 'min_data_in_leaf': 202, 'lambda_l1': 4.416715454062167e-05, 'lambda_l2': 97.7284456360912, 'max_bin': 73, 'num_leaves': 341},
{'learning_rate': 0.4360511696328724, 'feature_fraction': 0.9267135240558227, 'bagging_fraction': 0.8834671928769631, 'min_data_in_leaf': 426, 'lambda_l1': 0.00740756930750945, 'lambda_l2': 3.042623853713187, 'max_bin': 98, 'num_leaves': 508},
{'learning_rate': 0.4180174094678352, 'feature_fraction': 0.9626050915651176, 'bagging_fraction': 0.6880017140331517, 'min_data_in_leaf': 340, 'lambda_l1': 1.1359051360278207, 'lambda_l2': 0.0008619870627273151, 'max_bin': 82, 'num_leaves': 511},
{'learning_rate': 0.4540923930379812, 'feature_fraction': 0.971839776738276, 'bagging_fraction': 0.6672418499150679, 'min_data_in_leaf': 534, 'lambda_l1': 0.08926785662993982, 'lambda_l2': 2.835575484786637e-05, 'max_bin': 92, 'num_leaves': 509},
{'learning_rate': 0.3069006560814881, 'feature_fraction': 0.9963388008472429, 'bagging_fraction': 0.8793632084474166, 'min_data_in_leaf': 217, 'lambda_l1': 1.1982777608084696, 'lambda_l2': 26.943766780064788, 'max_bin': 80, 'num_leaves': 461}
]

In [12]:
params_list=[]
for param, model_init in zip(columns_list, param_model_init_list):
    param['model']['init'] = model_init
    params_list.append(param)

In [13]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), params_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=True, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [14]:
df_trial = pd.DataFrame(mytrial)
df_trial['trn_val_metric_diff_rate'] = df_trial['trn_val_metric_diff'] / df_trial['train_metric_mean']
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'trn_val_metric_diff_rate', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,trn_val_metric_diff_rate,message
0,2019-07-26 06:51:02.525527,modeling for 0,79,0.565624,0.759047,0.193423,0.341964,modeling for 0
1,2019-07-26 06:53:58.411834,modeling for 3,60,0.133805,0.172319,0.038513,0.287831,modeling for 3
2,2019-07-26 06:54:15.055231,modeling for 1,59,0.235309,0.383964,0.148656,0.631747,modeling for 1
3,2019-07-26 06:55:02.893601,modeling for 4,60,0.135884,0.193940,0.058056,0.427247,modeling for 4
4,2019-07-26 07:02:24.233145,modeling for 2,69,0.296226,0.338835,0.042609,0.143840,modeling for 2
5,2019-07-26 07:08:11.937239,modeling for 6,60,0.131612,0.178229,0.046617,0.354200,modeling for 6
6,2019-07-26 07:19:27.434836,modeling for 5,60,0.292319,0.345351,0.053032,0.181417,modeling for 5
7,2019-07-26 07:20:40.785668,modeling for 7,50,0.078629,0.118826,0.040198,0.511234,modeling for 7


In [15]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.35991783672867195
0 0.7590474878687717
3 0.17231878566746606
1 0.38396436444209325
4 0.1939399422227173
2 0.33883481507251795
6 0.17822949150590633
5 0.3453507182440888
7 0.11882614607970599


In [38]:
idx=0.34
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)*y_std+y_mean
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)